In [295]:
import pandas as pd
import numpy as np
# Cargar los datos desde el archivo CSV
df = pd.read_csv('HR RAW DATA.csv')

# Mostrar las primeras filas del DataFrame para entender la estructura de los datos


In [296]:
# Compruebo el numero de filas y columnas
df.shape

(1614, 42)

In [297]:
# Reviso que el Df no tenga duplicados
df.duplicated().sum()

0

Los titulos de todas las columnas del df se cambiaran de manera conjunta

ANALISIS DE COLUMNAS DE LA 21 A LA 30 PRIMERO REALIZO UNA PRIMERA VISUALIZACION DEL CONJUNTO

In [298]:
df_columnas_21_30 = df.iloc[:, 21:31]
df_columnas_21_30

,NUMCOMPANIESWORKED,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TOTALWORKINGYEARS,TrainingTimesLastYear
0,7,Y,No,13,"3,0",3,NaN,0,NaN,5
1,0,NaN,NaN,14,"3,0",1,NaN,1,"34,0",5
2,1,NaN,No,11,"3,0",4,NaN,0,"22,0",3
3,3,Y,NaN,19,"3,0",2,NaN,2,NaN,2
4,2,Y,No,12,"3,0",4,NaN,1,NaN,5
...,...,...,...,...,...,...,...,...,...,...
1609,1,Y,NaN,11,NaN,1,NaN,1,NaN,6
1610,1,NaN,NaN,12,"3,0",2,NaN,0,NaN,5
1611,0,NaN,No,18,"3,0",4,NaN,0,"9,0",3
1612,4,NaN,No,13,NaN,4,NaN,2,"12,0",3


In [299]:
df_columnas_21_30.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1614 entries, 0 to 1613
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   NUMCOMPANIESWORKED        1614 non-null   int64 
 1   Over18                    713 non-null    object
 2   OverTime                  938 non-null    object
 3   PercentSalaryHike         1614 non-null   int64 
 4   PerformanceRating         1419 non-null   object
 5   RelationshipSatisfaction  1614 non-null   int64 
 6   StandardHours             419 non-null    object
 7   StockOptionLevel          1614 non-null   int64 
 8   TOTALWORKINGYEARS         1088 non-null   object
 9   TrainingTimesLastYear     1614 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 126.2+ KB


In [300]:
df_columnas_21_30.dtypes

NUMCOMPANIESWORKED           int64
Over18                      object
OverTime                    object
PercentSalaryHike            int64
PerformanceRating           object
RelationshipSatisfaction     int64
StandardHours               object
StockOptionLevel             int64
TOTALWORKINGYEARS           object
TrainingTimesLastYear        int64
dtype: object

In [301]:
# Resumen estadístico de las columnas numéricas count no nulas
df_columnas_21_30.describe()

,NUMCOMPANIESWORKED,PercentSalaryHike,RelationshipSatisfaction,StockOptionLevel,TrainingTimesLastYear
count,1614.000000,1614.000000,1614.000000,1614.000000,1614.000000
mean,2.673482,15.165428,2.704461,0.791202,2.809789
std,2.506152,3.648610,1.079031,0.842396,1.297765
min,0.000000,11.000000,1.000000,0.000000,0.000000
25%,1.000000,12.000000,2.000000,0.000000,2.000000
50%,2.000000,14.000000,3.000000,1.000000,3.000000
75%,4.000000,18.000000,4.000000,1.000000,3.000000
max,9.000000,25.000000,4.000000,3.000000,6.000000


In [302]:
# count = no nulos, unique = cantidad de valor unico, Top= valor mas comun  freq=frecuencia del valor mas comun
df_columnas_21_30.describe(include = "object").T

,count,unique,top,freq
Over18,713,1,Y,713
OverTime,938,2,No,682
PerformanceRating,1419,2,"3,0",1205
StandardHours,419,1,"80,0",419
TOTALWORKINGYEARS,1088,40,"10,0",144


1.  NumCompaniesWorked: Número de compañías en las que el empleado ha trabajado."NUMCOMPANIESWORKED" entero sin datos nulos, la mayoria de valores estan el rango de 0 a 4 compañias donde el empleado ha trabajado (79%), y el resto en 5, 6, 7, 8 y 9 compañías. No hay valores negativos.  No realizamos cambios


In [303]:
df_columnas_21_30["NUMCOMPANIESWORKED"].unique()


array([7, 0, 1, 3, 2, 4, 8, 9, 5, 6])

In [304]:

df_columnas_21_30["NUMCOMPANIESWORKED"].value_counts()

1    573
0    226
3    169
4    157
2    156
7     84
6     73
5     66
9     59
8     51
Name: NUMCOMPANIESWORKED, dtype: int64

2.  Over18: Indica si el empleado es mayor de 18 años. "Over18"- Revisar porque el 56% de los datos son NaN para personas con mas de 18 años.  Etiquetamos los desconocidos.

In [305]:
df_columnas_21_30['Over18'] = df_columnas_21_30['Over18'].replace({'Y': True, 'No': False, np.nan: "Desconocido" })

In [306]:
df_columnas_21_30['Over18'].unique()

array([True, 'Desconocido'], dtype=object)

In [307]:
df_columnas_21_30["Over18"].unique()

array([True, 'Desconocido'], dtype=object)

In [308]:
df_columnas_21_30["Over18"].value_counts()

Desconocido    901
True           713
Name: Over18, dtype: int64

3. OverTime: Indica si el empleado trabaja horas extras ("Yes" para sí o "No" para no)."OverTime".  En este caso, el 58% de los datos tienen si o no y dentro de esta muestra diría que en principio  el 73 % han trabajado horas extras y el 27% no. etiquetamos los datos desconocidos.


In [309]:
df_columnas_21_30["OverTime"].value_counts()

No     682
Yes    256
Name: OverTime, dtype: int64

In [310]:
df_columnas_21_30['OverTime'] = df_columnas_21_30['OverTime'].replace({'Yes': True, 'No': False, np.nan: "Desconocido" })

In [311]:
df_columnas_21_30["OverTime"].value_counts()

False          682
Desconocido    676
True           256
Name: OverTime, dtype: int64

4. PercentSalaryHike: El porcentaje de aumento salarial del empleado."PercentSalaryHike", int no nulos, no hay valores negativos asi que solo cambiamos el tipo de datos a float para realizar operaciones con otras columnas.

In [312]:
df_columnas_21_30["PercentSalaryHike"].value_counts()

11    232
13    230
12    225
14    220
15    110
18     98
17     88
16     86
19     82
20     60
22     59
21     51
23     29
24     25
25     19
Name: PercentSalaryHike, dtype: int64

In [313]:
# Como estamos trabajando con float la convierto de int a float 
df_columnas_21_30['PercentSalaryHike'] = df_columnas_21_30['PercentSalaryHike'].astype(float)

In [314]:
df_columnas_21_30['PercentSalaryHike'].astype

<bound method NDFrame.astype of 0       13.0
1       14.0
2       11.0
3       19.0
4       12.0
        ... 
1609    11.0
1610    12.0
1611    18.0
1612    13.0
1613    13.0
Name: PercentSalaryHike, Length: 1614, dtype: float64>

5.  PerformanceRating: Calificación de rendimiento del empleado. PerformanceRating reemplazamos "," por ". "y etiquetamos desconocidos.convertimos a float excepto desconocidos

In [444]:
df_columnas_21_30 = df.iloc[:, 21:31]

In [445]:
df_columnas_21_30["PerformanceRating"].unique()

array(['3,0', '4,0', nan], dtype=object)

In [446]:
df_columnas_21_30["PerformanceRating"].value_counts()

3,0    1205
4,0     214
Name: PerformanceRating, dtype: int64

In [447]:
df_columnas_21_30['PerformanceRating'] = df_columnas_21_30['PerformanceRating'].str.replace(',', '.')

In [448]:
# Reemplazar NaN con una etiqueta específica para los valores desconocidos
df_columnas_21_30['PerformanceRating'].fillna('Desconocido', inplace=True)

In [449]:
df_columnas_21_30["PerformanceRating"].value_counts()

3.0            1205
4.0             214
Desconocido     195
Name: PerformanceRating, dtype: int64

In [451]:
df_columnas_21_30["PerformanceRating"].unique()

array(['3.0', '4.0', 'Desconocido'], dtype=object)

In [452]:
# Convertir los datos a tipo float, excepto los valores desconocidos
df_columnas_21_30['PerformanceRating'] = pd.to_numeric(df_columnas_21_30['PerformanceRating'], errors='coerce')

In [453]:
df_columnas_21_30["PerformanceRating"].astype

<bound method NDFrame.astype of 0       3.0
1       3.0
2       3.0
3       3.0
4       3.0
       ... 
1609    NaN
1610    3.0
1611    3.0
1612    NaN
1613    3.0
Name: PerformanceRating, Length: 1614, dtype: float64>

In [454]:
df_columnas_21_30["PerformanceRating"].value_counts()

3.0    1205
4.0     214
Name: PerformanceRating, dtype: int64

6- RelationshipSatisfaction: Nivel de satisfacción en las relaciones interpersonales del empleado.RelationshipSatisfaction, en principio dejaria como int porque son numeros ordinales que se refieren a niveles de satisfaccion de los empleados. Niveles de 1-4

In [455]:
df_columnas_21_30["RelationshipSatisfaction"].unique()

array([3, 1, 4, 2])

In [456]:
df_columnas_21_30["RelationshipSatisfaction"].value_counts()

3    504
4    468
2    339
1    303
Name: RelationshipSatisfaction, dtype: int64

7. StandardHours: Las horas estándar de trabajo.StandardHours, Entiendo que las horas estandar son 80 aunque tenemos muchos desconocidos. Convertimos a float y etiquetamos desconocidos.

In [459]:
df_columnas_21_30["StandardHours"].unique()

array([nan, '80,0'], dtype=object)

In [461]:
df_columnas_21_30["StandardHours"].value_counts()

80,0    419
Name: StandardHours, dtype: int64

In [464]:
# Reemplazar comas por puntos
df_columnas_21_30['StandardHours'] = df_columnas_21_30['StandardHours'].str.replace(',', '.')


# Convertir los datos a tipo float, excepto los valores desconocidos
df_columnas_21_30['StandardHours'] = pd.to_numeric(df_columnas_21_30['StandardHours'], errors='coerce')

# Reemplazar NaN por "Desconocido"
df_columnas_21_30['StandardHours'].fillna('Desconocido', inplace=True)


In [465]:
df_columnas_21_30["StandardHours"].value_counts()

Desconocido    1195
80.0            419
Name: StandardHours, dtype: int64

8 StockOptionLevel: Nivel de opciones de compra de acciones del empleado. StockOptionLevel. int y en principio lo dejaria de esta manera. no hay nulos.

In [466]:
df_columnas_21_30["StockOptionLevel"].value_counts()

0    687
1    666
2    172
3     89
Name: StockOptionLevel, dtype: int64

In [467]:
df_columnas_21_30["StockOptionLevel"].unique()

array([0, 1, 2, 3])

9. TotalWorkingYears: Total de años de experiencia laboral del empleado.TOTALWORKINGYEARS, convertimos a float y etiquetamos los desconocidos. Tenemos el 67,4% de datos para poder realizar analisis.

In [468]:
df_columnas_21_30["TOTALWORKINGYEARS"].unique()

array([nan, '34,0', '22,0', '28,0', '20,0', '21,0', '33,0', '40,0',
       '18,0', '25,0', '15,0', '17,0', '26,0', '16,0', '24,0', '14,0',
       '23,0', '27,0', '19,0', '11,0', '38,0', '37,0', '13,0', '12,0',
       '29,0', '10,0', '36,0', '35,0', '9,0', '31,0', '32,0', '8,0',
       '7,0', '30,0', '6,0', '5,0', '4,0', '3,0', '2,0', '1,0', '0,0'],
      dtype=object)

In [469]:
df_columnas_21_30['TOTALWORKINGYEARS'] = df_columnas_21_30["TOTALWORKINGYEARS"].str.replace(',', '.')

In [470]:
# Convertir los datos a tipo float, excepto los valores desconocidos
df_columnas_21_30['TOTALWORKINGYEARS'] = pd.to_numeric(df_columnas_21_30['TOTALWORKINGYEARS'], errors='coerce')

# Reemplazar NaN con una etiqueta específica para los valores desconocidos
df_columnas_21_30['TOTALWORKINGYEARS'].fillna('Desconocido', inplace=True)

In [471]:
df_columnas_21_30["TOTALWORKINGYEARS"].value_counts()

Desconocido    526
10.0           144
8.0             86
6.0             84
9.0             69
5.0             66
7.0             56
4.0             54
1.0             53
12.0            34
3.0             32
13.0            30
14.0            30
11.0            29
16.0            28
15.0            28
20.0            28
18.0            27
21.0            23
17.0            22
2.0             21
22.0            18
19.0            17
24.0            14
23.0            13
28.0            13
0.0              8
26.0             8
36.0             6
29.0             6
25.0             6
33.0             6
37.0             5
27.0             5
31.0             4
35.0             3
32.0             3
30.0             3
40.0             3
34.0             2
38.0             1
Name: TOTALWORKINGYEARS, dtype: int64

10. TrainingTimesLastYear: Número de veces que el empleado recibió capacitación el año pasado.TrainingTimesLastYear. int no hay nulos

In [472]:
df_columnas_21_30["TrainingTimesLastYear"].unique()

array([5, 3, 2, 0, 1, 4, 6])

In [473]:
df_columnas_21_30["TrainingTimesLastYear"].value_counts()

2    598
3    534
4    137
5    136
1     77
6     72
0     60
Name: TrainingTimesLastYear, dtype: int64

Resumen de la información obtenida de la primera visualización de las columnas
1 - La mayoria de emleados han trabajado en otras compañias entre = 0 y 4 años, 79% (columna NUMCOMPANIESWORKED). Sin cambios
2 - El 56% de datos de edad > 18 años es NaN (Columna Over18), verificar esta columna, todos deberian ser mayores de 18 años True y etiquetamos desconocidos
3-  En este caso "han tabajado horas extras", el 58% de los datos tienen si o no y dentro de esta muestra diría que en principio  el 73 % han trabajado horas extras y el 27% no Etiquetamos desconocidos.
4- El porcentaje de aumento salarial tiene una media de 15,16% cambiamos a float
5-PerformanceRating: Calificación de rendimiento del empleado valores entre 3 y 4 etiquetamos los desconocidos.
6. RelationshipSatisfaction Niveles de satisfacción ente 1-4 en principio los dejamos como int
7. Horas estandar consideramos 80 porque es el unico valor que nos aportan y los demas son desconocidos. Este es el resultado de la primera exploración
8. Niveles de opcion de compra de 0-3 en principio lo dejamos como int.
9. Total años de experiencia contamos con el 67 % de datos y convertimos a float los datos y etiquetamos los desconocidos.
10. Los empleados han recibido de 0 a 6 capacitaciones. int